In [3]:
import pathlib
import textwrap
import os

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
GOOGLE_API_KEY=os.getenv('API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [6]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


## Model Selection

In [7]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

In [ ]:
def construct_prompt(conversation_history, user_query, is_initial_query):
    """
    Construct a prompt that emulates the expertise and responsibilities of a Google Assistant Product Manager,
    focusing on specified categories and indicating the potential for broader assistance in the future.
    """
    if is_initial_query:
        prompt_intro = (
            "You are an AI acting as a Technical Product Manager for Google Assistant, "
            "specializing in evaluating new feature requests, strategizing new product developments, "
            "and conducting root cause analysis/hypothesis testing for products and metrics. "
            "Currently, you're adept at handling inquiries related to these three main areas, "
            "but you're also capable of adapting and expanding your expertise to cover a wider range of topics in the future.\n\n"
        )
    else:
        prompt_intro = (
            "As you understand, the intent of the conversations can be categorized into new feature requests, strategizing new product developments, "
            "and conducting root cause analysis/hypothesis testing for products and metrics. Use this categorization to ask clarifying questions, "
            "following the approach of any Technical Product Manager, until you have all the necessary information for the task. If the user's intent "
            "doesn't fall under any of these categories, simply provide the requested information and ONLY FOR THIS CASE introduce your persona and your role.\n\n"
            "[Please adhere strictly to a maximum of 3-4 questions for the clarifying questions, asking them one at a time to ensure clarity.]"
            "\n\n"
        )

    prompt = prompt_intro + conversation_history + f"\nBusiness Professional: {user_query}\nProduct Manager AI:"
    return prompt

In [ ]:
def get_conversation_response(conversation_history, user_query, is_initial_query):
    """
    Generate a response that offers strategic advice, further questions, or insights,
    leveraging the expertise of a Google Assistant Product Manager.
    """
    prompt = construct_prompt(conversation_history, user_query, is_initial_query)
    model = genai.GenerativeModel('gemini-1.5-pro-latest')
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            temperature=0.7)
    )
    return response.text

In [ ]:
def main():
    print("Connecting you to your AI-modeled Google Assistant Product Manager...")
    introduction = (
        "Hello, I'm designed to perform as a Google Assistant Product Manager, equipped to assist with "
        "new feature requests, product development strategies, and metrics analysis. These are my core areas "
        "of expertise. However, I'm continuously learning and expanding my capabilities to include a broader "
        "spectrum of topics in the future. How can I assist you today?"
    )
    print(introduction)
    
    conversation_history = ""
    is_initial_query = True

    while True:
        user_query = input("Business Professional: ").strip()
        if user_query.lower() == 'exit':
            print("Thank you for our discussion today. I look forward to assisting you further in the future!")
            break

        ai_response = get_conversation_response(conversation_history, user_query, is_initial_query)
        
        # Update conversation history
        conversation_history += f"\nBusiness Professional: {user_query}\nProduct Manager AI: {ai_response}"
        is_initial_query = False
        
        print(f"\nBusiness Professional: {user_query}")
        print("\n")
        print(f"\nProduct Manager AI: {ai_response}")
        print("#"*100)